<a href="https://colab.research.google.com/github/clovisdanielss/Colab-Projects/blob/main/mnist-fashion/MNIST_Fashion_Tensorflow_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>